In [1]:
!pip install keras_applications
!pip install transformers

     |████████████████████████████████| 50 kB 739 kB/s             


In [2]:
import numpy as np 
import pandas as pd 
import os
import re
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing import image
import time
from nltk.corpus import wordnet as wn
from PIL import Image
import requests
import torch
from transformers import ViTFeatureExtractor, ViTModel, ViTForImageClassification
import cv2
import matplotlib.pyplot as plt

In [3]:
in_path = '../input/dataset-crawler-hyponyms/'
data_dict = {}
for path, dirs, files in os.walk(in_path):
    for i in dirs:
        for path2, dirs2, files2 in os.walk('../input/dataset-crawler-hyponyms/'+i):
            urls = [in_path+i+'/'+j for j in files2]
            data_dict[str(i)] = urls
print(len(data_dict))

10


In [4]:
data_dict.keys()

dict_keys(['tabby cat', 'angora cat', 'lynx cat', 'siamese cat', 'tiger cat', 'persian cat', 'cougar cat', 'leopard cat', 'egyptian cat', 'cat'])

# Classifiers

In [5]:
def evaluate(pred_class, actual_class, acc, TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions):
    # find wordnet synsets for predicted class 
    wn_preds = wn.synsets(pred_class, pos=wn.NOUN)
    wn_pred=wn_preds[0]
    
    # find synset of ground truth class
    try:            
        actual_class=actual_class.replace(' ', '_')
        wn_label = wn.synsets(actual_class, pos=wn.NOUN)[0]
    except:   # we have added the word 'cat' for disambiguation (e.g. tiger cat vs tiger)
        actual_class = actual_class.replace('_cat', '')
        wn_label = wn.synsets(actual_class, pos=wn.NOUN)[0]
    #print('Actual:', wn_label, actual_class)
    #print('Pred:', wn_preds, pred_class)
    
    # Find hypernyms of ground truth class and predicted class 
    try:
        label_hyper_actual = wn_label.hypernyms()[0]
    except: # if there are no hypernyms, get entity as the global hypernym
        label_hyper_actual = wn.synsets('entity')[0]
    try:
        label_hyper_pred = wn_pred.hypernyms()[0]
    except:
        label_hyper_pred = wn.synsets('entity')[0]
    #print('Hypernyms:', label_hyper_pred, label_hyper_actual)
    
    # check if predicted class is a sub-class of actual label 
    # first, find all hyponyms (in any taxonomy depth) of the ground truth class
    label_hypo = list(set([w for s in wn_label.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    
    # then, find all hyponym synsets
    synset_hypo = [wn.synsets(word, pos=wn.NOUN)[0] for word in label_hypo]
    
    # if predicted is subset, then prediction is correct
    hypo_intersection=set(wn_preds).intersection(set(synset_hypo))
    same=False
    
    # if there is an overlap in actual/pred synsets or label names
    if hypo_intersection or (wn_label in wn_preds) or (actual_class.lower()==pred_class.lower()):
        #print('Same')
        acc+=1
        TP+=1
        same=True
        
    # if actual/pred synsets have a common immediate parent node (or entity as the common parent)
    elif (label_hyper_pred==label_hyper_actual) and not(label_hyper_pred==wn.synset('entity.n.01') and label_hyper_actual==wn.synset('entity.n.01')):
        #print('Siblings. Hypernyms:', label_hyper_pred)
        FN+=1
        siblings+=1
        common_misconceptions.append((actual_class, pred_class))
        
    # otherwise, ground truth and pred classes have more than 1-level difference
    else:
        #print('Not immediately related')
        FN+=1
        common_misconceptions.append((actual_class, pred_class))
     
    if not same:   # find WordNet-based metrics for different ground truth/pred synsets
        min_hypernym = wn_pred.lowest_common_hypernyms(wn_label)
        #print('Lowest common hypernym: ', min_hypernym)
    
        shortest_path = wn_pred.path_similarity(wn_label)
        sp_list.append(shortest_path)
        #print('Shortest path similarity between {} and {}: {}'.format(pred_class, actual_class, shortest_path))
    
        # hortest path and the maximum depth of the taxonomy
        lch = wn_pred.lch_similarity(wn_label)
        lch_list.append(lch)
        #print('Leacock-Chodorow similarity between {} and {}: {}'.format(pred_class, actual_class, lch))
    
        # depth of the two synsets in the taxonomy and that of their Least Common Subsumer (most specific ancestor node)
        wups = wn_pred.wup_similarity(wn_label)
        wups_list.append(wups)
        #print('Wu-Palmer similarity between {} and {}: {}'.format(pred_class, actual_class, wups))
    
    return acc, TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions

In [6]:
def predictions(image, model, feature_extractor, all_time, preds):
    start = time.time()    
    inputs = feature_extractor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    elapsed_time = time.time()-start
    probs = logits.softmax(dim=1)
    idx = torch.topk(probs, 3, largest=True, sorted=True, out=None)
    top3pos = idx[1][0]
    pred_classes = {}
    top_counter = 1
    for item in top3pos:
        predicted_class_idx = item.item()
        pred_classes[top_counter] = model.config.id2label[predicted_class_idx]
        top_counter+=1
    
    # model predicts one of the 1000 ImageNet classes
    preds.append(pred_classes)
    all_time.append(elapsed_time)
    
    return all_time, preds, pred_classes

In [7]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained("google/vit-base-patch16-224")
model_name = 'ViT'

Downloading:   0%|          | 0.00/160 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

In [8]:
results={}
all_misconceptions = {}
for data_name, data in data_dict.items():
    all_time = []
    preds = []
    acc=0
    TP=0
    FN=0
    siblings=0
    common_misconceptions = []
    sp_list, lch_list, wups_list = [], [], []

    for url in data:
        image = cv2.imread(url)
        #try:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        all_time, preds, top = predictions(image, model, feature_extractor, all_time, preds)
        top_1_class=list(top.values())[0]
        if ',' in top_1_class:
            splt = top_1_class.split(',')
            top_1_class = splt[0]
            top_1_class=top_1_class.rstrip().lstrip()
        if ' ' in top_1_class:
            top_1_class=top_1_class.replace(' ', '_')
        acc, TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions = evaluate(top_1_class, data_name, acc, 
                                                             TP, FN, sp_list, lch_list, wups_list, siblings, common_misconceptions)

        #except Exception as e:
        #    print(str(e))
        mean_acc = acc/(TP+FN)
        #print(mean_acc, TP, FN, np.mean(sp_list), np.mean(lch_list), np.mean(wups_list), np.sum(all_time), siblings)
        run_id = data_name+'_'+model_name
        results[run_id] = (data_name, model_name, mean_acc, TP, FN, np.mean(sp_list), 
                               np.mean(lch_list), np.mean(wups_list), np.sum(all_time), siblings)
        all_misconceptions[run_id] = common_misconceptions

/opt/conda/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/conda/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
all_df = pd.DataFrame(results.values(), columns=['dataset', 'model', 'accuracy', 'TP', 'FN', 'path_similarity', 'LCH', 'WUPS', 'predict_time', 'siblings'])
all_df

,dataset,model,accuracy,TP,FN,path_similarity,LCH,WUPS,predict_time,siblings
0,tabby cat,ViT,0.420000,21,29,0.251492,2.153414,0.807684,28.302018,15
1,angora cat,ViT,0.020000,1,49,0.307726,2.386669,0.828446,27.367108,44
2,lynx cat,ViT,0.900000,45,5,0.047619,0.593064,0.090909,27.425049,0
3,siamese cat,ViT,0.880000,44,6,0.150000,1.365357,0.435462,27.110064,2
4,tiger cat,ViT,0.100000,6,54,0.145514,1.541567,0.664677,32.151742,0
5,persian cat,ViT,0.920000,46,4,0.270833,2.192400,0.761513,27.240068,3
6,cougar cat,ViT,0.940000,47,3,0.150877,1.583148,0.685897,26.595741,0
7,leopard cat,ViT,0.000000,0,50,0.173397,1.760817,0.749496,26.823442,1
8,egyptian cat,ViT,0.500000,25,25,0.123812,1.415148,0.526685,26.442018,2
9,cat,ViT,0.851852,23,4,0.062586,0.849353,0.356268,15.093332,0


In [10]:
all_df.to_pickle('./cats_imagenet_ViT.pkl')

In [11]:
import pickle 

with open('./ViT_all_common_misconceptions.pkl', 'wb') as f:
    pickle.dump(all_misconceptions, f)
        
with open('./ViT_all_common_misconceptions.pkl', 'rb') as f:
    loaded_dict = pickle.load(f)